# Data Preparation for the Nord_H2ub Spine Model

This jupyter notebook contains all routines for the preparation of the input data sources into a input data file for the model in Spine. 

**Authors:** Johannes Giehl (jfg.eco@cbs.dk)

## Import of packages

In [1]:
import numpy as np
import pandas as pd

## File paths

In [15]:
#set path to correct folders

excel_file_path = '../Input_data/'

In [16]:
#set name of the relevant files

PV_data_filename = 'PV_availability_factors_Kasso-v001-djh_2023_12_15.xlsx'

## Workflow of the data preparation

- data import
- data adjustments
- final data settings
- excel/csv export


In [17]:
df_PV_values = pd.read_excel(excel_file_path+PV_data_filename)

In [18]:
df_PV_values

,# Units: time in UTC,local_time in Europe/Copenhagen,electricity in kW,irradiance_direct in unknown,irradiance_diffuse in unknown,temperature in ÂºC,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,"# {""units"": {""time"": ""UTC""","""local_time"": ""Europe/Copenhagen""","""electricity"": ""kW""","""irradiance_direct"": ""unknown""","""irradiance_diffuse"": ""unknown""","""temperature"": ""\u00baC""}","""params"": {""local_time"": true","""header"": true","""lat"": ""55.0368""","""lon"": ""9.2338""","""date_from"": ""2019-01-01""","""date_to"": ""2019-12-31""","""dataset"": ""merra2""","""capacity"": ""1""","""system_loss"": ""0""","""tracking"": ""0""","""tilt"": ""35""","""azim"": ""180""","""raw"": true}}"
1,time,local_time,electricity,irradiance_direct,irradiance_diffuse,temperature,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01 00:00:00,2019-01-01 01:00:00,0,0,0,7.426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01 01:00:00,2019-01-01 02:00:00,0,0,0,7.517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-01 02:00:00,2019-01-01 03:00:00,0,0,0,7.611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8757,2019-12-31 19:00:00,2019-12-31 20:00:00,0,0,0,1.257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8758,2019-12-31 20:00:00,2019-12-31 21:00:00,0,0,0,1.102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8759,2019-12-31 21:00:00,2019-12-31 22:00:00,0,0,0,1.114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8760,2019-12-31 22:00:00,2019-12-31 23:00:00,0,0,0,1.217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
